## Basic Convolution From Scratch

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

In [2]:
import numpy as np
import sys
import torch.nn as nn
import torch

In [3]:
# create the filter
l1_filter = np.random.randn(5,3,3,3) # 5 3D filters


In [4]:
import copy
def conv_(imgs, filter):
    assert(imgs.shape[0] == imgs.shape[1] and filter.shape[0] == filter.shape[1] and imgs.shape[-1] == filter.shape[-1])
    
    # remainding space - e.g. if remainder==2, then the first two columns of the filter will convolve with the last two columns of the image 
    remainder = imgs.shape[0] % filter.shape[0]
    k = filter.shape[0]

    feature_map = np.zeros((imgs.shape[0] - k + 1, imgs.shape[0] - k + 1))

    for i in range(len(feature_map)):
        f = copy.copy(filter)
        for j in range(len(feature_map[1])):
            feature = np.sum(np.multiply(imgs[i:i+k, j:j+k, :], f))
            feature_map[i, j] = feature

    return feature_map

In [5]:
def conv(imgs, conv_filter):
    if len(imgs.shape) > 2 or len(conv_filter.shape) > 3: 
        if imgs.shape[-1] != conv_filter.shape[-1]:
            print('ERROR: Number of channels (dimension [-1]) in the image and filter must match')
            sys.exit()
    if conv_filter.shape[1] != conv_filter.shape[2]: 
        print('ERROR: filter must be a square matrix. Dims [1] and [2] must match')
    
    if conv_filter.shape[-1] % 2 == 0:
        print('ERROR: filter size must be odd')
    
    feature_maps = np.zeros((imgs.shape[0] - conv_filter.shape[1] + 1, 
                            imgs.shape[1] - conv_filter.shape[1] + 1, 
                            conv_filter.shape[0]))
    
    for filter_num in range(conv_filter.shape[0]):
        curr_filter = conv_filter[filter_num, :] # get one filter from the stack of filters
        conv_map = conv_(imgs, curr_filter)
        feature_maps[:, :, filter_num] = conv_map

    return feature_maps



In [6]:
test_img = np.random.randn(222, 222, 3)
test_filter = l1_filter[0]

In [7]:
test_out = conv(test_img, l1_filter)

In [8]:
test_out.shape

(220, 220, 5)

In [9]:
unfold = nn.Unfold(3, 1, 0, 1)
item = torch.randn(2, 2, 3, 3) 
out = unfold(item)
out.shape

torch.Size([2, 18, 1])

## Basic Involution (using some Pytorch)

```
B: batch size, H: height, W: width
C: channel number, G: group number
K: kernel size, s: stride, r: reduction ratio
```

In [ ]:
def inv(imgs, r=1, kernel_size=3, group_ch=1, stride=1, padding=0):
    '''
    assumes imgs is a tensor of shape (batch_size, channels, w, h)
    '''
    
    k = kernel_size
    c_1 = imgs.shape[1]
    g = c_1 // group_ch

    o = nn.AvgPool2d(stride, stride) if stride > 1 else nn.Identity()

    # these conv layers are what will be changed w/ backprop
    reduce = nn.Conv2d(c_1, c_1 // r, 1) 
    span = nn.Conv2d(c_1 // r, g*k*k, k, padding=padding) # is k supposed to be here??? 
    weights = (o(imgs))
    weights = reduce(weights)
    
    weights = span(weights)
    
    b, c_2, h, w = weights.shape
    
    weights = weights.view(b, g, 1, k**2, w*h)
    
    patches = nn.Unfold(k, padding=padding)(imgs) # patches.shape = (b, c*k*k, L) where L is the number of times a 3x3 kernel can shift over an image
    assert(patches.shape == (b, c_1*k*k, patches.shape[-1]))
    
    patches = patches.view(b, g, group_ch, k**2, w*h) # ( c_1 // group_ch) * group_ch = c_1  #  x_unfolded = x_unfolded.view(b, self.groups, self.group_ch, self.kernel_size**2, h, w)

    out = (patches*weights) # out.shape = (b, g, g//c, k*k, w*h)

    # out should be shape (b, c, h, w) 
    out = out.view(b, c_1, k*k, w*h) # we replace (g, g//c) with c because c=g*(g//c)
    out = out.sum(dim=2) 
    out = out.view(b, c_1, w, h)

    return out


    

    

In [51]:
test_img = torch.randn(1, 3, 220, 220)


test_out = inv(test_img)
test_out.shape

torch.Size([1, 3, 218, 218])